In [ ]:
#실행할 때마다 수정
num = 3
set_output_dir = f'outputs-base-{num}/'
# set_model_dir = f'outputs-base-{num-1}/checkpoint-4669'
set_jsonl = f'test_output-base-{num}.jsonl'


set_threshold = 0.5
set_epoch = 5
set_lr = 4e-5

#raberta-large 모델은 batch size가 32이하여야 colab 'T4 GPU'환경에서 작동

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/nlp/data

/content/drive/MyDrive/nlp/data


In [ ]:
cat nikluge-ea-2023-dev.jsonl nikluge-ea-2023-train.jsonl > nikluge-ea-2023-merge.jsonl

In [ ]:
ls

label2id.json                 nikluge-ea-2023-train.jsonl
nikluge-ea-2023-dev.jsonl     outputs-base-1/
nikluge-ea-2023-merge.jsonl   test_output-base-1.jsonl
nikluge-ea-2023-sample.jsonl  wandb/
nikluge-ea-2023-test.jsonl


In [ ]:
#install here
!pip3 install torch==1.13.0.* transformers==4.31.0 accelerate==0.21.0 datasets==2.14.0 wandb==0.13.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.1/890.1 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 84.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.2/492.2 kB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 73.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 68.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 102.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# import here
import json
import datasets
import os
from tqdm import tqdm
from transformers import (AutoTokenizer,
                          AutoModelForSequenceClassification,
                          TrainingArguments,
                          Trainer,
                          EvalPrediction,
                        )
import torch
import numpy as np
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from torch.utils.data import DataLoader, random_split

In [ ]:
seed = 2023
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

In [ ]:
#load dataset
merge_ds = datasets.Dataset.from_json("nikluge-ea-2023-merge.jsonl")
test_ds = datasets.Dataset.from_json("nikluge-ea-2023-test.jsonl")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
labels = list(merge_ds["output"][0].keys())
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
with open(os.path.join('', "label2id.json"), "w") as f:
    json.dump(label2id, f)

In [ ]:
#define model & tokenizer & args
#If you want roberta-large model, replace roberta-base to reberta-large
# I think It's better to train base model first than use large model directly.

tokenizer = AutoTokenizer.from_pretrained("klue/roberta-base")


model = AutoModelForSequenceClassification.from_pretrained(
    'klue/roberta-base',
    problem_type = "multi_label_classification",
    num_labels = len(labels),
    id2label = id2label,
    label2id = label2id,
    max_length = 128,)

In [ ]:
args = TrainingArguments(
        output_dir=set_output_dir,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        learning_rate=set_lr,
        per_device_train_batch_size=64,
        per_device_eval_batch_size=64,
        num_train_epochs=set_epoch,
        weight_decay=0.01,
        load_best_model_at_end=True,
        metric_for_best_model= "f1",
        )

In [ ]:
#I checked max_length of data is 206.
def preprocess_data(examples):
        # take a batch of texts
        text1 = examples["input"]["form"]
        text2 = examples["input"]["target"]["form"]
        a = text1.replace("!!!!!", "!!")
        b = a.replace("...","..")
        c = b.replace("기혀니♡ 기혀니♡ ","기혀니♡ ")
        d = c.replace("&others&","")
        # encode them
        encoding = tokenizer(d, text2, padding="max_length", truncation=True, max_length=128)
        # add labels
        encoding["labels"] = [0.0] * len(labels)
        for key, idx in label2id.items():
            if examples["output"][key] == 'True':
                encoding["labels"][idx] = 1.0

        return encoding

In [ ]:
encoded_mds = merge_ds.map(preprocess_data, remove_columns = merge_ds.column_names)

Map:   0%|          | 0/42683 [00:00<?, ? examples/s]

In [ ]:
encoded_mds

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 42683
})

In [ ]:
# for i in tqdm(encoded_mds['input_ids']):
#   if len(i) != 128:
#     print("Error")
#     break

In [ ]:
# cont = []
# max = 0
# for i,data in tqdm(enumerate(encoded_mds['input_ids'])):
#   if len(data)>=128:
#     cont.append(i)
#   if max < len(data):
#     max = len(data)

In [ ]:
# for i in cont:
#   print(merge_ds['input'][i])
# print(max)
# len(cont)

In [ ]:
# modified_sentence = []
# for i in cont:
#   sentence = merge_ds['input'][i]['form']
#   a = sentence.replace("!!", "!")
#   b = a.replace("..",".")
#   c = b.replace("기혀니♡ 기혀니♡ ","기혀니♡ ")
#   d = c.replace("&others&","")
#   merge_ds['input'][i]['form'] = d
#   print(d)
#   print(merge_ds['input'][i]['form'])

### 전처리 완료

In [ ]:
def multi_label_metrics(predictions, labels, threshold=0.5):
        # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
        sigmoid = torch.nn.Sigmoid()
        probs = sigmoid(torch.Tensor(predictions))
        # next, use threshold to turn them into integer predictions
        y_pred = np.zeros(probs.shape)
        y_pred[np.where(probs >= threshold)] = 1
        # finally, compute metrics
        y_true = labels
        f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
        roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
        accuracy = accuracy_score(y_true, y_pred)
        # return as dictionary
        metrics = {'f1': f1_micro_average,
                   'roc_auc': roc_auc,
                   'accuracy': accuracy}
        return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    result = multi_label_metrics(predictions=preds, labels=p.label_ids, threshold = set_threshold)
    return result

In [ ]:
_, validation_dataset, _ = random_split(encoded_mds, [len(encoded_mds)-6400, 6400, 0])

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset = encoded_mds,
    eval_dataset = validation_dataset,
    tokenizer = tokenizer,
    compute_metrics = compute_metrics
)

In [ ]:
model.train()
torch.set_grad_enabled(True)
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,0.018000,0.019488,0.978769,0.986529,0.959375
2,0.021300,0.015983,0.982307,0.988577,0.965313
3,0.015100,0.008968,0.992079,0.994525,0.984062
4,0.011000,0.006549,0.994774,0.996673,0.990000


Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,0.018000,0.019488,0.978769,0.986529,0.959375
2,0.021300,0.015983,0.982307,0.988577,0.965313
3,0.015100,0.008968,0.992079,0.994525,0.984062
4,0.011000,0.006549,0.994774,0.996673,0.990000
5,0.008700,0.004802,0.996611,0.997857,0.993437


TrainOutput(global_step=3335, training_loss=0.014933706580967024, metrics={'train_runtime': 4388.2522, 'train_samples_per_second': 48.633, 'train_steps_per_second': 0.76, 'total_flos': 1.403871771706368e+16, 'train_loss': 0.014933706580967024, 'epoch': 5.0})

In [ ]:
device = torch.device('cuda:0')

In [ ]:
with open(os.path.join("label2id.json")) as f:
    label2id = json.load(f)
labels = list(label2id.keys())
id2label = {}
for k, v in label2id.items():
    id2label[v] = k

In [ ]:
def test_preprocess_data(examples):
        # take a batch of texts
        text1 = examples["input"]["form"]
        text2 = examples["input"]["target"]["form"]
        a = text1.replace("!!", "!")
        b = a.replace("..",".")
        c = b.replace("기혀니♡ 기혀니♡ ","기혀니♡ ")
        d = c.replace("&others&","")
        # encode them
        encoding = tokenizer(d, text2, padding="max_length", truncation=True, max_length=128)
        return encoding

In [ ]:
encoded_tds = test_ds.map(test_preprocess_data, remove_columns = test_ds.column_names).with_format("torch")
data_loader = DataLoader(encoded_tds, batch_size = 64)

Map:   0%|          | 0/4748 [00:00<?, ? examples/s]

In [ ]:
outputs = []
sigmoid = torch.nn.Sigmoid()

In [ ]:
with torch.no_grad():
  model.to(device)
  model.eval()
  torch.set_grad_enabled(False)
  for batch in tqdm(data_loader):
    oup = model(
        batch["input_ids"].to(device),
        token_type_ids = batch["token_type_ids"].to(device),
        attention_mask = batch["attention_mask"].to(device)
    )
    oup.logits

    probs = sigmoid(oup.logits).cpu().detach().numpy()
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= set_threshold)] = 1

    outputs.extend(y_pred.tolist())

  torch.set_grad_enabled(True)


100%|██████████| 75/75 [00:31<00:00,  2.36it/s]


In [ ]:
def jsonlload(fname):
    with open(fname, "r", encoding="utf-8") as f:
        lines = f.read().strip().split("\n")
        j_list = [json.loads(line) for line in lines]

    return j_list


def jsonldump(j_list, fname):
    with open(fname, "w", encoding='utf-8') as f:
        for json_data in j_list:
            f.write(json.dumps(json_data, ensure_ascii=False)+'\n')

In [ ]:
j_list = jsonlload("nikluge-ea-2023-test.jsonl")
for idx, oup in enumerate(outputs):
    j_list[idx]["output"] = {}
    for jdx, v in enumerate(oup):
        if v:
            j_list[idx]["output"][id2label[jdx]] = "True"
        else:
            j_list[idx]["output"][id2label[jdx]] = "False"

jsonldump(j_list, set_jsonl)